In [202]:
import gspread
from google.oauth2.service_account import Credentials
import pandas as pd
import itertools

In [203]:
# -------------------------------
# CONFIG
# -------------------------------
SPREADSHEET_NAME = "Stain Remover Variable Rounds"
SOURCE_SHEET = "Variables"
TARGET_SHEET = "combinations"
CREDENTIALS_FILE = "credentials.json"

In [204]:
# -------------------------------
# AUTH
# -------------------------------
scopes = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive",
]

creds = Credentials.from_service_account_file(
    CREDENTIALS_FILE, scopes=scopes
)

gc = gspread.authorize(creds)

In [205]:
# -------------------------------
# LOAD DATA
# -------------------------------
sh = gc.open(SPREADSHEET_NAME)
ws = sh.worksheet(SOURCE_SHEET)

# Read into DataFrame
data = ws.get("A1:I")
headers = data[0]
rows = data[1:]

df = pd.DataFrame(rows, columns=headers)

In [206]:
# -------------------------------
# PREP VARIABLES
# -------------------------------
columns = df.columns.tolist()

values_per_column = []

for col in columns:
    values = (
        df[col]
        .dropna()                    # remove NaN / None
        .astype(str)
        .str.strip()                 # remove whitespace-only
    )

    values = values[values != ""]    # remove empty strings

    values_per_column.append(values.tolist())

In [207]:
# -------------------------------
# CARTESIAN PRODUCT
# -------------------------------
combinations = list(itertools.product(*values_per_column))

combo_df = pd.DataFrame(combinations, columns=columns)

In [208]:
# -------------------------------
# WRITE TO "combinations" SHEET
# -------------------------------
# Delete sheet if it exists
try:
    sh.del_worksheet(sh.worksheet(TARGET_SHEET))
except gspread.exceptions.WorksheetNotFound:
    pass

combo_ws = sh.add_worksheet(
    title=TARGET_SHEET,
    rows=str(len(combo_df) + 1),
    cols=str(len(columns)),
)

combo_ws.update(
    [combo_df.columns.tolist()] +
    combo_df.values.tolist()
)

print(f"Created {len(combo_df)} combinations.")

Created 360 combinations.


In [209]:
# -------------------------------
# BOLD HEADER ROW
# -------------------------------
sheet_id = combo_ws._properties["sheetId"]

sh.batch_update({
    "requests": [
        {
            "repeatCell": {
                "range": {
                    "sheetId": sheet_id,
                    "startRowIndex": 0,
                    "endRowIndex": 1,   # only header row
                },
                "cell": {
                    "userEnteredFormat": {
                        "textFormat": {
                            "bold": True
                        }
                    }
                },
                "fields": "userEnteredFormat.textFormat.bold",
            }
        }
    ]
})

# -------------------------------
# FREEZE HEADER ROW
# -------------------------------
sheet_id = combo_ws._properties["sheetId"]

sh.batch_update({
    "requests": [
        {
            "updateSheetProperties": {
                "properties": {
                    "sheetId": sheet_id,
                    "gridProperties": {
                        "frozenRowCount": 1
                    }
                },
                "fields": "gridProperties.frozenRowCount"
            }
        }
    ]
})

# -------------------------------
# FILTER HEADER ROW
# -------------------------------
sheet_id = combo_ws._properties["sheetId"]

sh.batch_update({
    "requests": [
        {
            "setBasicFilter": {
                "filter": {
                    "range": {
                        "sheetId": sheet_id,
                        "startRowIndex": 0,
                        "startColumnIndex": 0,
                        "endColumnIndex": len(columns),
                    }
                }
            }
        }
    ]
})

{'spreadsheetId': '1u-dGqajNE5dMKJx_3DkalcdxPBTY8d5rPlRIogi7r0M',
 'replies': [{}]}

In [210]:
len(combo_df)

360